In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
from motion_integration import (integrate_motion, time_step, eps_picker, find_bond_forces, 
                                repulsive_force, evaluate_motion_equations)
from force_test import assemble_quad_matrix
from resistance_matrix_test import generate_resistance_matrices

In [ ]:
def get_tstep_distribution(expt_name):
    data_dir = '/Users/andrewwork/thesis/reg_stokeslets/3d/data/'
    data = np.load(data_dir + expt_name + '.npz')
    diffs = data['t'][1:] - data['t'][:-1]
    return np.bincount(np.round(-np.log2(diffs / .002)).astype('int'))

### Time steps in the experiments with stiff bonds

In [ ]:
t_steps = get_tstep_distribution('bd_expt089')
print(t_steps)

In [ ]:
t_steps2 = get_tstep_distribution('bd_expt092')
print(t_steps2)

### Forces  and torques in stable configuration

In [ ]:
n_nodes = 8
a, b = 1.5, 0.5
domain  = 'wall'
eps = eps_picker(n_nodes, a, b)
precompute_array = None
proc = 1
shear_vec = True
kappa = 25000.
lam = 0.1

In [ ]:
data = np.load('data/bd_expt092.npz')
ii = np.argwhere((data['t'] >= 0.35) * (data['t'] <= 0.36))[0][0]
distance = data['x'][ii]
center = np.array([data['x'][ii], data['y'][ii], data['z'][ii]])
rmat = data['r_matrices'][..., ii]
receptors = data['receptors']
bonds = data['bond_array'][..., ii]
print(data['t'][ii])
print(data['t'][ii-1])

In [ ]:
data['bond_array'][..., ii-2]

In [ ]:
theta = np.arctan2(rmat[2, 0], rmat[1, 0])
phi = np.arccos(rmat[0, 0])

In [ ]:
result = generate_resistance_matrices(
    eps, n_nodes, a, b, domain, distance, theta, 
    phi, shear_vec, proc, precompute_array
)

t_matrix, p_matrix, pt_matrix, r_matrix, shear_f, shear_t = result
res_matrix = np.block([[t_matrix, p_matrix], [pt_matrix, r_matrix]])

In [ ]:
bd_forces, bd_torques = find_bond_forces(receptors, bonds, center, rmat, kappa, lam, one_side=True)
point, rep_force = repulsive_force(center[0], rmat[:, 0])
point = np.array([point[0], point[1] + center[1], point[2] + center[2]])
rep_forces = np.array([rep_force, 0, 0])
rep_torques = np.cross([point[0] - center[0], point[1], point[2]],
                        [rep_force, 0, 0])
forces, torques = rep_forces + bd_forces, rep_torques + bd_torques

In [ ]:
print(bd_forces)
print(bd_torques)

In [ ]:
gen_forces = np.block([[shear_f - forces.reshape((3, 1))], 
                       [shear_t - torques.reshape((3, 1))]])
gen_vels = np.linalg.solve(res_matrix, -gen_forces)
trans_vels, rot_vels = np.squeeze(gen_vels[:3]), np.squeeze(gen_vels[3:])

In [ ]:
print(gen_forces.T)
print(trans_vels)
print(rot_vels)

In [ ]:
print(np.dot(res_matrix, gen_vels)[3:] + shear_t 
      + np.cross((np.dot(res_matrix, gen_vels)[:3] + shear_f).T, point - center).T)

print(bd_torques + np.cross(bd_forces, point - center))

In [ ]:
point - center

In [ ]:
print(forces - shear_f.T)
print(torques - shear_t.T)

In [ ]:
print(t_matrix)

In [ ]:
shear_t

In [ ]:
np.dot(res_matrix, gen_vels) + gen_forces